In [41]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [15]:
spark = SparkSession.builder.appName('CrimePrediction').getOrCreate()
df = spark.read.format("csv").load("../../datasets/FIR_Details_Data.csv",header=True,inferSchema=True)

In [16]:
df.columns

['District_Name',
 'UnitName',
 'FIRNo',
 'RI',
 'Year',
 'Month',
 'Offence_From_Date',
 'Offence_To_Date',
 'FIR_Reg_DateTime',
 'FIR_Date',
 'FIR Type',
 'FIR_Stage',
 'Complaint_Mode',
 'CrimeGroup_Name',
 'CrimeHead_Name',
 'Latitude',
 'Longitude',
 'ActSection',
 'IOName',
 'KGID',
 'IOAssigned_Date',
 'Internal_IO',
 'Place of Offence',
 'Distance from PS',
 'Beat_Name',
 'Village_Area_Name',
 'Male',
 'Female',
 'Boy',
 'Girl',
 'Age 0',
 'VICTIM COUNT',
 'Accused Count',
 'Arrested Male',
 'Arrested Female',
 'Arrested Count\tNo.',
 'Accused_ChargeSheeted Count',
 'Conviction Count',
 'FIR_ID',
 'Unit_ID',
 'Crime_No']

In [17]:
df.count()

1694191

In [18]:
df.groupBy('CrimeGroup_Name').count().show()

+--------------------+------+
|     CrimeGroup_Name| count|
+--------------------+------+
|      BURGLARY - DAY|  8328|
|      FALSE EVIDENCE|   259|
|Karnataka State L...| 91769|
|      UNNATURAL SEX |    52|
|ESCAPE FROM LAWFU...|   340|
|             SUICIDE|  6145|
|      PUBLIC JUSTICE|    31|
|      COUNTERFEITING|   320|
|      MISSING PERSON|126237|
|Attempting to com...|  1524|
|         CYBER CRIME| 79756|
|            ELECTION|  2671|
|           FOREIGNER|   431|
|CRIMINAL INTIMIDA...| 18092|
|OFFENCES AGAINST ...| 10072|
|      ARMS ACT  1959|  1665|
|ATTEMPT TO CULPAB...|   287|
|        ADULTERATION|   158|
|WRONGFUL RESTRAIN...|  3501|
|CULPABLE HOMICIDE...|   673|
+--------------------+------+
only showing top 20 rows



In [19]:
d_split = pyspark.sql.functions.split(df['FIR_Date'],'/')
df =df.withColumn('Date',d_split.getItem(0))
df = df.withColumn('Date', df['Date'].cast(IntegerType()))

## Replacing District Values

In [20]:
df = df.na.replace(['Belagavi City', 'Belagavi Dist'], ['Belagavi', 'Belagavi'], 'District_Name')
df = df.na.replace(['Bengaluru City', 'Bengaluru Dist','ISD Bengaluru','CID'], ['Bengaluru', 'Bengaluru','Bengaluru', 'Bengaluru'], 'District_Name')
df = df.na.replace(['Kalaburagi City'], ['Kalaburagi'], 'District_Name')
df = df.na.replace(['Hubballi Dharwad City'], ['Dharwad'], 'District_Name')
df = df.na.replace(['K.G.F'], ['Kolar'], 'District_Name')
df = df.na.replace(['Mangaluru City'], ['Mangaluru'], 'District_Name')

In [21]:
df.select('District_Name').distinct().show()

+--------------------+
|       District_Name|
+--------------------+
|             Ballari|
|            Bagalkot|
|            Belagavi|
|           Bengaluru|
|               Bidar|
|      Chamarajanagar|
|      Chickballapura|
|      Chikkamagaluru|
|Coastal Security ...|
|          Davanagere|
|    Dakshina Kannada|
|         Chitradurga|
|             Dharwad|
|              Haveri|
|               Gadag|
|              Hassan|
|  Karnataka Railways|
|              Kodagu|
|          Kalaburagi|
|               Kolar|
+--------------------+
only showing top 20 rows



In [22]:
df.select('FIR Type').distinct().show()

+-----------+
|   FIR Type|
+-----------+
|    Heinous|
|Non Heinous|
|       NULL|
+-----------+



In [23]:
df = df.na.replace(['NULL'], ['Not Applicable'], 'FIR Type')
df.select('FIR Type').distinct().show()

+--------------+
|      FIR Type|
+--------------+
|       Heinous|
|   Non Heinous|
|Not Applicable|
+--------------+



In [37]:
df.printSchema()

root
 |-- District_Name: string (nullable = true)
 |-- UnitName: string (nullable = true)
 |-- FIRNo: string (nullable = true)
 |-- RI: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Offence_From_Date: timestamp (nullable = true)
 |-- Offence_To_Date: string (nullable = true)
 |-- FIR_Reg_DateTime: timestamp (nullable = true)
 |-- FIR_Date: string (nullable = true)
 |-- FIR Type: string (nullable = true)
 |-- FIR_Stage: string (nullable = true)
 |-- Complaint_Mode: string (nullable = true)
 |-- CrimeGroup_Name: string (nullable = true)
 |-- CrimeHead_Name: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- ActSection: string (nullable = true)
 |-- IOName: string (nullable = true)
 |-- KGID: string (nullable = true)
 |-- IOAssigned_Date: string (nullable = true)
 |-- Internal_IO: integer (nullable = true)
 |-- Place of Offence: string (nullable = true)
 |-- Distance fro

In [49]:
model_columns = df.select(['District_Name','UnitName','Date','Month','Year','FIR Type','CrimeGroup'])

In [46]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

District_NameIdx = StringIndexer(inputCol='District_Name', 
                               outputCol='District_Index') 
DistrictEncode = OneHotEncoder(inputCol='District_Index', 
                               outputCol='District_Vec') 

UnitNameIdx = StringIndexer(inputCol='UnitName', 
                               outputCol='UnitName_Index') 
UnitNameEncode = OneHotEncoder(inputCol='UnitName_Index', 
                               outputCol='UnitName_Vec') 

Fir_TypeIdx = StringIndexer(inputCol='FIR Type', 
                               outputCol='FirType_Index') 
Fir_TypeEncode = OneHotEncoder(inputCol='FirType_Index', 
                               outputCol='FirType_Vec') 

In [36]:
# encoding predicting column[CrimeGroup_Name]

indexer = StringIndexer(inputCol="CrimeGroup_Name", outputCol="CrimeGroup")
indexerModel = indexer.fit(df)
df = indexerModel.transform(df)


In [39]:
df = df.withColumn('CrimeGroup', df['CrimeGroup'].cast(IntegerType()))
df.select('CrimeGroup').distinct().show()

+----------+
|CrimeGroup|
+----------+
|        31|
|        85|
|        65|
|        53|
|        78|
|        34|
|        81|
|        28|
|        76|
|        26|
|        27|
|        44|
|        12|
|        91|
|        22|
|        47|
|         1|
|        52|
|        13|
|         6|
+----------+
only showing top 20 rows



In [29]:
assembler = VectorAssembler(inputCols=['District_Vec','UnitName_Vec','Date','Month','Year','FirType_Vec'],outputCol='features')

In [47]:
#modeling
log_reg = LogisticRegression(featuresCol='features',labelCol='CrimeGroup')

pipe = Pipeline(stages=[District_NameIdx,UnitNameIdx,Fir_TypeIdx,DistrictEncode,UnitNameEncode,Fir_TypeEncode,assembler,log_reg])

In [51]:
# Splitting the data into train and test 
train_data, test_data = model_columns.randomSplit([0.7, .3]) 
  
# Fitting the model on training data 
fit_model = pipe.fit(train_data) 
  
# Storing the results on test data 
results = fit_model.transform(test_data) 
  
# Showing the results 
results.show() 

+-------------+----------+----+-----+----+-----------+----------+--------------+--------------+-------------+---------------+------------------+-------------+--------------------+--------------------+--------------------+----------+
|District_Name|  UnitName|Date|Month|Year|   FIR Type|CrimeGroup|District_Index|UnitName_Index|FirType_Index|   District_Vec|      UnitName_Vec|  FirType_Vec|            features|       rawPrediction|         probability|prediction|
+-------------+----------+----+-----+----+-----------+----------+--------------+--------------+-------------+---------------+------------------+-------------+--------------------+--------------------+--------------------+----------+
|     Bagalkot|Amengad PS|   1|    2|2021|Non Heinous|         0|          24.0|         583.0|          0.0|(33,[24],[1.0])|(1054,[583],[1.0])|(2,[0],[1.0])|(1092,[24,616,108...|[7.03067100942534...|[0.12399598868599...|       5.0|
|     Bagalkot|Amengad PS|   1|    6|2016|Non Heinous|        36|   

In [53]:
# Importing the evaluator 
from pyspark.ml.evaluation import BinaryClassificationEvaluator 
  
# Calling the evaluator 
res = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='CrimeGroup') 
  
ROC_AUC = res.evaluate(results) 
ROC_AUC

0.7284088646210262